In [ ]:
from matplotlib.pyplot import imshow
import matplotlib.cm as cm
import matplotlib.pylab as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import PIL
from PIL import ImageFilter
import cv2
import itertools
import random
import tensorflow as tf
import imutils
from imutils import paths
import os
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D , UpSampling2D ,Conv2DTranspose
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

%matplotlib inline

In [ ]:
# Check GPU (skip this if GPU hardware acceleration is not chosen)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
def preprocess(img):
  return cv2.GaussianBlur(img, (3, 3), 0)

In [ ]:
IMAGE_SHAPE = (105, 105)
data_path = "/content/gdrive/MyDrive/deepfont_data_normal/"
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
  #rotation_range=30,
	zoom_range=0.2,
	width_shift_range=0.1,
	height_shift_range=0.1,
	#shear_range=0.2,
	#horizontal_flip=True,
  #preprocessing_function=preprocess,
  validation_split=0.2,
  rescale=1/255
)
image_train = image_generator.flow_from_directory(directory=data_path, color_mode="grayscale", target_size=IMAGE_SHAPE, subset="training")
image_val = image_generator.flow_from_directory(directory=data_path, color_mode="grayscale", target_size=IMAGE_SHAPE, subset="validation")

In [ ]:
 def create_model():
  model=Sequential()

  # Cu Layers 
  model.add(Conv2D(64, kernel_size=(48, 48), activation='relu', input_shape=(105,105,1)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(128, kernel_size=(24, 24), activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2DTranspose(128, (24,24), strides = (2,2), activation = 'relu', padding='same', kernel_initializer='uniform'))
  model.add(UpSampling2D(size=(2, 2)))

  model.add(Conv2DTranspose(64, (12,12), strides = (2,2), activation = 'relu', padding='same', kernel_initializer='uniform'))
  model.add(UpSampling2D(size=(2, 2)))

  #Cs Layers
  model.add(Conv2D(256, kernel_size=(12, 12), activation='relu'))

  model.add(Conv2D(256, kernel_size=(12, 12), activation='relu'))

  model.add(Conv2D(256, kernel_size=(12, 12), activation='relu'))

  model.add(Flatten())

  model.add(Dense(4096, activation='relu'))

  model.add(Dropout(0.5))

  model.add(Dense(4096,activation='relu'))

  model.add(Dropout(0.5))

  model.add(Dense(2383,activation='relu'))

  model.add(Dense(10, activation='softmax'))
 
  return model

In [ ]:
batch_size = 128
epochs = 100
model= create_model()
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(), 
    optimizer=optimizers.SGD(learning_rate=0.001, decay=1e-6, momentum=0.9, nesterov=True), 
    #optimizer=Adam(learning_rate=0.01),
    metrics=['accuracy']
)

In [ ]:
early_stopping=callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=0, mode='min')

filepath="/content/gdrive/MyDrive/deepfont_model_normal.h5"

checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [early_stopping,checkpoint]

In [ ]:
model.fit(
    image_train, shuffle=True,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=image_val,
    callbacks=callbacks_list
)